# Projekt numer 4 - operacje na ramkach danych  

Wiktor Woźniak

## Wstęp
  
Ten projekt ma na celu odtworzenie pięciu różnych zapytań SQL w Python za pomocą pakietów numpy oraz pandas. Każde zapytanie zostało wykonane na 2 sposoby: 
  
1. Zapytanie SQL  
2. Zapytanie Python  
  
Dodatkowo dla każdego zapytania zostały wykonane sprawdzenia poprawności wyników ramek oraz zostało sprawdzone czy wynikowy typ zapytań pythona jest ramką danych.  
  
Operuję na ramkach danych ze strony https://travel.stackexchange.com/. Dodatkowo w celu wyjaśnienia zapytań korzystam ze strony https://meta.stackexchange.com/questions/2677/database-schema-documentation-for-the-public-data-dump-and-sede/2678#2678 oraz https://ia600107.us.archive.org/27/items/stackexchange/readme.txt


## Przygotowanie danych
  
Zaimportowanie potrzebnych pakietów, zaczytanie plików, stworzenie bazy danych oraz dodanie do niej plików

In [1]:
import numpy as np
import pandas as pd
import os, os.path
import sqlite3
import time
pd.set_option("display.notebook_repr_html", False) # ustawienia wyświetlania

# Zaczyatnie plików


Badges = pd.read_csv("travel_stackexchange_com/Badges.csv.gz",
                    compression = 'gzip')
Badges.head()

Comments = pd.read_csv("travel_stackexchange_com/Comments.csv.gz",
                        compression = 'gzip')
Comments.head()

PostLinks = pd.read_csv("travel_stackexchange_com/PostLinks.csv.gz",
compression = 'gzip')
PostLinks.head()

Posts = pd.read_csv("travel_stackexchange_com/Posts.csv.gz",
compression = 'gzip')
Posts.head()

Tags = pd.read_csv("travel_stackexchange_com/Tags.csv.gz",
compression = 'gzip')
Tags.head()

Users = pd.read_csv("travel_stackexchange_com/Users.csv.gz",
compression = 'gzip')
Users.head()

Votes = pd.read_csv("travel_stackexchange_com/Votes.csv.gz",
compression = 'gzip')
Votes.head()

# Stworzenie bazy danych oraz dodnie plików
baza = os.path.join('baza.db')

if os.path.isfile(baza): 
    os.remove(baza)

conn = sqlite3.connect(baza) 
Badges.to_sql("Badges", conn)
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)




## Zadanie 1
  
Wybór etykiet oraz ich ilości, wtedy gdy ilość jest większa od 1000. Etykiety są posortowane malejąco po ilości.

### Zapytanie SQL

In [2]:
sql1 = pd.read_sql_query("""
                    SELECT Count, TagName
                    FROM Tags
                    WHERE Count > 1000
                    ORDER BY Count DESC
                    """, conn)

### Zapytanie Python

In [3]:
python1 = Tags.loc[ Tags.Count > 1000, ["Count", "TagName"]]\
            .sort_values(["Count"], ascending = [False])\
            .reset_index(drop = True)


### Sprawdzenie poprawności wyników 

In [4]:
sql1.equals(python1)

True

In [5]:
isinstance(python1, pd.core.frame.DataFrame)

True

## Zadanie 2
  
Wybór dziesięciu najczęściej wskazywanych przez użytkowników lokalizacji posortowanych malejąco po ilości.

### SQL

In [6]:
sql2 = pd.read_sql_query("""
                    SELECT Location, COUNT(*) AS Count
                    FROM (
                        SELECT Posts.OwnerUserId, Users.Id, Users.Location
                        FROM Users
                        JOIN Posts ON Users.Id = Posts.OwnerUserId
                    )
                    WHERE Location NOT IN ('')
                    GROUP BY Location
                    ORDER BY Count DESC
                    LIMIT 10
                    """, conn)


### Python

In [7]:
# Stworzenie złączonej ramki i wybór odpowiednich kolumn
polaczone = Users.merge(Posts, left_on = "Id", right_on = "OwnerUserId", how = "inner")\
                        .rename(columns = {"Id_x": "Id"})
polaczone = polaczone[["Id", "Location"]] 

# Wykonanie agregacji na powyższej ramce
python2 = polaczone.loc[polaczone.Location != "", ["Location"]]\
                        .groupby("Location")\
                        .size().reset_index().rename(columns={0: 'Count'})

# Formatowanie wyniku
python2 = python2.sort_values(["Count"], ascending = [False]).reset_index(drop = True)
python2 = python2.head(10)

### Sprawdzenie poprawności wyników

In [8]:
sql2.equals(python2)

True

In [9]:
isinstance(python2, pd.core.frame.DataFrame)

True

## Zadanie 3
  
Wybór ogólnej ilości złotych odznak (Class = 1) zdobytych w danym roku.

### SQL 

In [10]:
sql3 = pd.read_sql_query("""
                    SELECT Year, SUM(Number) AS TotalNumber
                    FROM (
                            SELECT
                            Name,
                            COUNT(*) AS Number,
                            STRFTIME('%Y', Badges.Date) AS Year
                            FROM Badges
                            WHERE Class = 1
                            GROUP BY Name, Year
                        )
                    GROUP BY Year
                    ORDER BY TotalNumber
                    """, conn)


### Python

In [11]:
# Stworzenie ramki i wybranie odpowiednich kolumn
wybor = Badges.loc[Badges.Class == 1, :].copy()
wybor.loc[:, ["Year"]] = pd.to_datetime(wybor['Date']).apply(lambda x: x.strftime('%Y'))
wybor = wybor[["Name", "Year"]].groupby(["Name", "Year"])\
        .size().reset_index().rename(columns={0: 'Number'})

# Wykonanie agregacji na powyższej ramce i formatowanie wyniku
python3 = wybor.groupby(["Year"])["Number"].agg(np.sum)\
        .reset_index().rename(columns = ({'Number': 'TotalNumber'})).sort_values(["TotalNumber"]).reset_index(drop = True)


### Sprawdzenie poprawności

In [12]:
sql3.equals(python3)

True

In [13]:
isinstance(python3, pd.core.frame.DataFrame)

True

## Zadanie 4
  
Wybieramy informację o dziesięciu urzytkowników których średnia odpowiedzi (PostTypeId = 2) jest największą i dodatkowo najmniejsze mają najniższe ID.

### SQL 

In [14]:
sql4 = pd.read_sql_query("""
                    SELECT
                        Users.AccountId,
                        Users.DisplayName,
                        Users.Location,
                        AVG(PostAuth.AnswersCount) as AverageAnswersCount
                    FROM
                    (
                        SELECT
                            AnsCount.AnswersCount,
                            Posts.Id,
                            Posts.OwnerUserId
                        FROM (
                                SELECT Posts.ParentId, COUNT(*) AS AnswersCount
                                FROM Posts
                                WHERE Posts.PostTypeId = 2
                                GROUP BY Posts.ParentId
                            ) AS AnsCount
                        JOIN Posts ON Posts.Id = AnsCount.ParentId
                    ) AS PostAuth
                    JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
                    GROUP BY OwnerUserId
                    ORDER BY AverageAnswersCount DESC, AccountId ASC
                    LIMIT 10

                    """, conn)


### Python

In [15]:
# Wykonanie operacji na odpowiedniej podramce ramki Posts
AnsCount = Posts.loc[Posts.PostTypeId == 2, ["ParentId"]].groupby(["ParentId"])\
            .size().reset_index().rename(columns = {0: "AnswersCount"})

# Złączenie ramek Posts i powyższej oraz wybór odpowiednich kolumn
PostAuth = Posts.merge(AnsCount, left_on = "Id", right_on = "ParentId", how = "inner")\
            .rename(columns = {"Id_x": "Id"})[["AnswersCount", "Id", "OwnerUserId"]]

#  Złączenie ramek Users i powyższej
python4 = PostAuth.merge(Users, left_on = "OwnerUserId", right_on = "AccountId", how = "inner")\
            .rename(columns = {"OwnerUserId_x": "OwnerUserId"})

# Wykonanie agregacji na odpowiedniej podramce powyższej ramki
python4 = python4.groupby(["AccountId", "DisplayName", "Location"], dropna = False)["AnswersCount"]\
            .agg(np.mean).reset_index().rename(columns = ({'AnswersCount': 'AverageAnswersCount'}))

# Formatowanie wyniku
python4 = python4.sort_values(["AverageAnswersCount", "AccountId"], ascending = [False, True]).head(10).reset_index(drop = True)

### Sprawdzenie poprawności wyników 

In [16]:
sql4.equals(python4)

True

In [17]:
isinstance(python4, pd.core.frame.DataFrame)

True

## Zadanie 5
  
Wybieramy informacje o 10 postach, dla których różnica głosów łącznie (zaakceptowanych przez twórce, pozytywnych i ulubionych) przed 2020 i po 2020 była największa, nowych głosów jest więcej niz starych i tytuł posta z tymi głosami nie jest pusty.

### SQL 

In [18]:
sql5 = pd.read_sql_query("""
                        SELECT Posts.Title, Posts.Id,
                            STRFTIME('%Y-%m-%d', Posts.CreationDate) AS Date,
                            VotesByAge.Votes
                        FROM Posts
                        JOIN (
                            SELECT
                                PostId,
                                MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
                                MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
                                SUM(Total) AS Votes
                            FROM (
                                SELECT
                                    PostId,
                                    CASE STRFTIME('%Y', CreationDate)
                                        WHEN '2021' THEN 'new'
                                        WHEN '2020' THEN 'new'
                                        ELSE 'old'
                                        END VoteDate,
                                    COUNT(*) AS Total
                                FROM Votes
                                WHERE VoteTypeId IN (1, 2, 5)
                                GROUP BY PostId, VoteDate
                            ) AS VotesDates
                            GROUP BY VotesDates.PostId
                            HAVING NewVotes > OldVotes
                        ) AS VotesByAge ON Posts.Id = VotesByAge.PostId
                        WHERE Title NOT IN ('')
                        ORDER BY Votes DESC
                        LIMIT 10
                        """, conn)

### Python

In [19]:
# Wybór odpowiednich wierszy z ramki Votes i dodanie kolumny, której wartość zależy od daty oddanie głosu
VotesDates = Votes.loc[Votes.VoteTypeId.isin([1,2,5]),:].copy()
VotesDates["VoteDate"] = pd.to_datetime(VotesDates['CreationDate']).apply(lambda x: x.strftime('%Y'))
VotesDates["VoteDate"] = VotesDates["VoteDate"].apply(lambda x: "new" if(x == '2020' or x =='2021') else "old")

# Wykonanie agregacji na powyższej ramce.
VotesDates = VotesDates.groupby(["PostId", "VoteDate"])\
            .size().reset_index().rename(columns = {0: "Total"})

# Dodanie do powyższej ramki dwóch nowych kolumn z wartościami zależnymi od tego czy oddany głos jest nowy
n = lambda x: 1 if x == 'new' else 0.
VotesDates["NewVotes"] = VotesDates.Total * VotesDates.VoteDate.map(n)
o = lambda x: 1 if x == 'old' else 0.
VotesDates["OldVotes"] = VotesDates.Total * VotesDates.VoteDate.map(o)

# Wykonanie dwóch operacji na powyższej ramce i złączenie wynikowych ramek
VotesByAgeMax = VotesDates.groupby(["PostId"])[["NewVotes", "OldVotes"]]\
                .agg(np.max).reset_index()
VotesByAgeTotal = VotesDates.groupby(["PostId"])[["Total"]].agg(np.sum)\
                .reset_index().rename(columns = {"Total": "Votes"})
VotesByAge = VotesByAgeMax.merge(VotesByAgeTotal, on = "PostId", how = 'inner')

# Wybór odpowiednich wierszy z powyższej ramki i złączenie ich z ramką Posts
VotesByAge = VotesByAge.loc[VotesByAge.NewVotes > VotesByAge.OldVotes,:]
python5 = VotesByAge.merge(Posts, left_on = "PostId", right_on = "Id", how = "inner", suffixes = ("",""))

# Wybór daty i dołączenie jej do powyższej ramki
python5["Date"] = pd.to_datetime(python5['CreationDate']).apply(lambda x: x.strftime('%Y-%m-%d'))

# Wybór odpowiednich kolumn i wierszy oraz formatowanie wyników
python5 = python5.loc[~python5.Title.isnull(), ["Title","Id","Date","Votes"]]\
        .sort_values(["Votes", "Title"], ascending = [False, True]).head(10).reset_index(drop = True)

### Sprawdzenie poprawności wyników

In [20]:
sql5.equals(python5)

True

In [21]:
isinstance(python5, pd.core.frame.DataFrame)

True

## Podsumowanie
  
Podsumowując zarówno ten projekt jak i poprzednie mogę stwierdzić, że w Pythonie wykonywanie operacji na ramkach dancyh jest dla mnie przyjemniejsze niż w R. Dodatkowo z moich obserwacji wynika, że zadanie 5, w którym jest dużo formatowania daty i funkcji case:when, najszybciej wykonuje się w Pythonie, a w R te czasy wykonania były zauważalnie wolniejsze. Co ciekawe sortowanie w numpy przebiega trochę inaczej niż normalnie w SQL lub w R w momencie gdy sortujemy po jakiejś wartości i ta wartość w dwóch wierszach wyjdzie taka sama to te wiersze mogą być zamienione kolejnośćią w porównaniu do SQL czy też R dlatego wtedy dobrze jeszcze dodatkowo posortować po jakiejść innej kolumnie ( najlepiej takiej, która ma unikatowe wartości), aby equals() zwracała True  

In [22]:
# Zamknięcie bazy danych
conn.close()